# 검증 세트 (validation set)
### 검증 세트의 필요성
	- 모델의 과대적합 또는 과소적합 판단에 도움을 줌
	- 테스트 세트 없이도 모델 성능 측정 가능함
### 데이터 분할 방법
	- 훈련 세트: 60%
	- 검증 세트: 20%
	- 테스트 세트: 20%
### 검증 및 테스트 세트의 크기
	- 일반적으로 전체 데이터의 20~30%를 사용함
	- 데이터가 충분히 많은 경우 더 적은 비율도 가능함
### 모델 평가 및 선택 과정
	1. 훈련 세트로 모델을 훈련함
	2. 검증 세트로 모델을 평가함
	3. 매개변수를 조정하고 최적 모델을 선택함
	4. 훈련 세트와 검증 세트를 합쳐 최종 모델을 훈련함
	5. 테스트 세트로 최종 성능을 평가함
**이 방법을 통해 실제 환경에서의 모델 성능을 예측할 수 있음**

In [5]:
import pandas as pd

# 'wine_csv_data' 파일을 읽어들여 DataFrame으로 저장
wine = pd.read_csv('https://bit.ly/wine_csv_data')

# 'class' 열을 타깃으로 사용하고 나머지 열은 특성 배열(data)에 저장
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()  # 특성 데이터
target = wine['class'].to_numpy()  # 타깃 데이터

# 훈련 세트의 입력 데이터와 타깃 데이터를 train_input과 train_target 배열에 저장
from sklearn.model_selection import train_test_split

# 전체 데이터를 훈련 세트(80%)와 테스트 세트(20%)로 분리
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

# 훈련 세트에서 다시 20%를 검증 세트로 분리
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

# 훈련 세트와 검증 세트의 크기 확인
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [6]:
from sklearn.tree import DecisionTreeClassifier

# DecisionTreeClassifier 객체 생성, 랜덤 시드 설정
dt = DecisionTreeClassifier(random_state=42)

# 결정 트리 모델을 훈련 세트(sub_input)와 타깃(sub_target)으로 학습
dt.fit(sub_input, sub_target)

# 훈련 세트 정확도 출력
print(dt.score(sub_input, sub_target))

# 검증 세트 정확도 출력
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


# 교차 검증(cross validation)
- 검증 세트를 떼어 내어 평가하는 과정을 여러번 반복 -> 점수를 평균하여 최종 검증 점수를 얻음
# 3-폴드 교차 검증
- 훈련 세트를 세 부분으로 나눠서 교차 검증을 수행하는 것을 말함
- 통칭 k-폴드 교차 검증(k-fold cross validation) 또는 k-겹 교차 검증이라고 하며, 훈련 세트를 몇 부분으로 나누냐에 따라 다르게 부름

In [7]:
from sklearn.model_selection import cross_validate

# 교차 검증을 수행하여 모델의 성능을 평가
# cross_validate()는 주어진 모델(dt)을 훈련 데이터(train_input)와 타깃(train_target)에 대해 교차 검증 수행
# 기본적으로 'fit_time', 'score_time', 'test_score' 등의 결과를 반환
scores = cross_validate(dt, train_input, train_target)

# 교차 검증 결과 출력
print(scores)

{'fit_time': array([0.01321316, 0.00742412, 0.00731111, 0.00625086, 0.00569081]), 'score_time': array([0.00129795, 0.00065994, 0.00061798, 0.00080109, 0.00047517]), 'test_score': array([0.87019231, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [8]:
# 검증 폴드에 담긴 5개의 점수를 평균하여 교차 검증의 최종 점수 출력
import numpy as np

# 'test_score'는 각 폴드에서의 검증 점수를 포함하므로 이를 평균하여 최종 점수 출력
print(np.mean(scores['test_score']))

0.8554925223957948


In [12]:
from sklearn.model_selection import StratifiedKFold

# StratifiedKFold 객체를 사용하여 계층화된 교차 검증을 수행
# StratifiedKFold는 데이터의 각 클래스 비율을 동일하게 유지하면서 교차 검증을 수행함
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())

# 검증 폴드에 담긴 점수를 평균하여 교차 검증의 최종 점수 출력
print(np.mean(scores['test_score']))

0.8554925223957948


In [13]:
# 훈련 세트를 섞은 후 10-폴드 교차 검증을 수행
# StratifiedKFold 객체 생성: n_splits=10으로 10개의 폴드로 나누고,
# shuffle=True로 데이터를 섞어서 교차 검증을 진행함, random_state=42로 결과의 재현성을 보장
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# cross_validate() 함수로 10-폴드 교차 검증 수행
scores = cross_validate(dt, train_input, train_target, cv=splitter)

# 검증 폴드에 담긴 점수를 평균하여 교차 검증의 최종 점수 출력
print(np.mean(scores['test_score']))

0.8581873425226026


# 하이퍼파라미터 튜닝
- 사용자 지정 파라미터
- 검증 세트의 점수나 교차 검증을 통해서 매개변수를 조금씩 바꾸면서 튜닝을 작업
- 사람의 개입 없이 하이퍼파라미터 튜닝을 자동으로 수행하는 기술을 'AutoML'이라 함

# 그리드 서치 (Grid Search)

그리드 서치는 머신러닝 모델의 하이퍼파라미터 튜닝 방법 중 하나입니다. 주어진 하이퍼파라미터 값의 조합을 모두 탐색하여 최적의 조합을 찾는 방식으로, 모든 가능한 하이퍼파라미터 값의 조합에 대해 모델을 학습시키고 평가합니다. 

## 주요 특징
- **모든 조합 탐색**: 그리드 서치는 지정한 모든 하이퍼파라미터 값의 조합을 시도해 최적의 조합을 찾습니다.
- **계산 비용**: 모든 경우를 시도하기 때문에 계산 비용이 높아질 수 있으며, 하이퍼파라미터의 수와 값의 범위가 클수록 시간이 많이 소요됩니다.
- **장점**: 간단하고 모든 경우의 수를 검토할 수 있어 확실한 최적화가 가능합니다.
- **단점**: 시간과 리소스를 많이 소비하여 비효율적일 수 있습니다.

## 사용 예시
사이킷런(Scikit-Learn) 라이브러리에서는 `GridSearchCV` 클래스를 사용하여 그리드 서치를 구현할 수 있습니다.

```python
from sklearn.model_selection import GridSearchCV

# 모델과 하이퍼파라미터 범위 설정
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf']
}
grid_search = GridSearchCV(SVC(), param_grid, cv=5)
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 확인
print("Best Parameters:", grid_search.best_params_)

In [15]:
from sklearn.model_selection import GridSearchCV

# 'min_impurity_decrease' 하이퍼파라미터의 값을 변경하며 그리드 서치를 수행할 파라미터 범위 정의
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

# GridSearchCV 객체 생성: 모델, 하이퍼파라미터 범위, n_jobs=-1로 모든 CPU 코어를 사용하여 계산 속도 향상
# DecisionTreeClassifier 모델을 사용하고, 하이퍼파라미터 'min_impurity_decrease' 값을 최적화
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

# 그리드 서치를 훈련 데이터에 대해 실행하여 최적의 하이퍼파라미터를 찾음
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [16]:
# 그리드 서치를 통해 찾은 최적의 모델을 best_estimator_ 속성에서 가져옴
dt = gs.best_estimator_

# 최적의 모델을 훈련 데이터에 대해 평가하여 정확도 출력
print(dt.score(train_input, train_target))

0.9615162593804117


In [17]:
# 그리드 서치를 통해 최적화된 하이퍼파라미터를 출력
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [19]:
# 그리드 서치에서 각 하이퍼파라미터 조합에 대한 평균 테스트 점수를 출력
print(gs.cv_results_['mean_test_score'])

[0.86800067 0.86453617 0.86492226 0.86780891 0.86761605]


In [20]:
# 'mean_test_score'에서 가장 높은 점수를 얻은 하이퍼파라미터 조합의 인덱스를 찾음
best_index = np.argmax(gs.cv_results_['mean_test_score'])

# 해당 인덱스를 사용하여 최적의 하이퍼파라미터 값을 출력
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


1. 먼저 탐색할 매개변수를 지정
2. 그 다음 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾음. 이 조합은 그리드 서치 객체에 저장
3. 그리드 서치는 최상의 매개변수에서(교차 검증에 사용한 훈련 세트가 아니라) 전체 훈련 세트를 사용해 최종 모델을 훈련. 이 모델도 그리드 서치 객체에 저장

In [27]:
# 탐색할 하이퍼파라미터 범위 정의
# 'min_impurity_decrease', 'max_depth', 'min_samples_split'의 값을 각각 지정
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
        'max_depth': range(5, 20, 1),
        'min_samples_split': range(2, 100, 10)}

# GridSearchCV 객체 생성: 하이퍼파라미터 탐색을 위해 모든 CPU 코어를 사용하도록 n_jobs=-1 설정
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

# 그리드 서치를 훈련 데이터에 대해 실행하여 최적의 하이퍼파라미터를 찾음
gs.fit(train_input, train_target)

# 최상의 하이퍼파라미터 조합 출력
print(gs.best_params_)

# 최상의 교차 검증 점수 출력
# mean_test_score에서 가장 높은 값 출력
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 14, 'min_impurity_decrease': np.float64(0.0004), 'min_samples_split': 12}
0.8683865773302731


/Users/leesung-joon/jupyter_env/lib/python3.13/site-packages/numpy/ma/core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


# 랜덤 서치 (Random Search)

랜덤 서치는 하이퍼파라미터 최적화 기법으로, 검색 공간에서 하이퍼파라미터 조합을 무작위로 샘플링하여 최적의 값을 찾는 방법입니다. 주요 특징은 다음과 같습니다:

- **효율성**: 전체 조합을 탐색하는 그리드 서치와 달리 무작위로 탐색하여 시간과 비용이 절약됩니다.
- **성능 개선**: 특정 하이퍼파라미터가 중요한 경우, 랜덤 서치로 더 빠르게 성능이 좋은 조합을 찾을 수 있습니다.
- **활용 사례**: 대규모 하이퍼파라미터 튜닝 시 유용하며, 신경망 최적화 등에 자주 사용됩니다.

랜덤 서치는 다양한 조합을 탐색하여, 복잡한 모델의 하이퍼파라미터를 효율적으로 튜닝할 수 있는 기법입니다.

In [33]:
from scipy.stats import uniform, randint  # uniform은 실숫값을, randint는 정수값을 뽑는 분포

# randint를 사용하여 0에서 10 사이의 정수 10개를 랜덤하게 추출
rgen = randint(0, 10)
rgen.rvs(10)  # 10개의 랜덤 정수 값을 반환

array([3, 4, 1, 0, 1, 1, 4, 5, 8, 1])

In [34]:
# rgen.rvs(1000)으로 0과 10 사이의 랜덤 정수 1000개를 추출한 후, np.unique()로 고유한 값과 각 값의 출현 빈도 계산
# return_counts=True로 각 값이 나온 횟수를 함께 반환
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 98, 115, 100, 112, 103,  86,  95,  90, 100, 101]))

In [35]:
# uniform(0, 1)을 사용하여 0과 1 사이에서 균등 분포를 따르는 랜덤 실수 10개를 추출
ugen = uniform(0, 1)
ugen.rvs(10)  # 0과 1 사이의 실수 10개를 랜덤으로 반환

array([0.80839819, 0.49096945, 0.96250961, 0.8180152 , 0.0857778 ,
       0.44014344, 0.54136916, 0.7344938 , 0.5245174 , 0.06005187])

In [40]:
# 하이퍼파라미터 범위 정의:
# 'min_impurity_decrease'는 uniform 분포로 0.0001에서 0.001 사이의 값,
# 'max_depth', 'min_samples_split', 'min_samples_leaf'는 randint 분포로 각각 다른 범위에서 정수값을 생성
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
        'max_depth': randint(20, 50),
        'min_samples_split': randint(2, 25),
        'min_samples_leaf': randint(1, 25)}

from sklearn.model_selection import RandomizedSearchCV

# RandomizedSearchCV 객체 생성:
# n_iter=100으로 100번의 랜덤 샘플링을 통해 하이퍼파라미터를 탐색,
# n_jobs=-1로 가능한 모든 CPU 코어를 사용하여 계산 속도를 높임
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)

# RandomizedSearchCV를 훈련 데이터에 대해 실행하여 최적의 하이퍼파라미터를 찾음
gs.fit(train_input, train_target)

# 최적의 하이퍼파라미터 조합 출력
print(gs.best_params_)

# 최고의 교차 검증 점수 출력
# 'mean_test_score'에서 가장 높은 값 출력
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 39, 'min_impurity_decrease': np.float64(0.00034102546602601173), 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884


In [41]:
# 그리드 서치 또는 랜덤 서치에서 찾은 최적의 모델을 best_estimator_ 속성에서 가져옴
dt = gs.best_estimator_

# 최적의 모델을 테스트 데이터에 대해 평가하여 정확도 출력
print(dt.score(test_input, test_target))

0.86
